#import библиотек

In [ ]:
import torch
import torchvision
import os
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from torchvision.transforms import v2
import glob
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from IPython.display import clear_output
import tqdm
import pandas as pd
from torchvision import datasets
from torch.utils.data import DataLoader, Subset

In [ ]:
device = 'cuda'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#loading train data

In [ ]:
!unzip '/content/drive/MyDrive/journey-springfield.zip'

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: train/simpsons_dataset/moe_szyslak/pic_1025.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1026.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1027.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1028.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1029.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1030.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1031.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1032.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1033.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1034.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1035.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1036.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1037.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pic_1038.jpg  
  inflating: train/simpsons_dataset/moe_szyslak/pi

In [ ]:
path = "train/simpsons_dataset"
data_paths = glob.glob(f"{path}/*/*.jpg")
data_labels = [path.split('/')[-2] for path in data_paths]
train_path, test_path = train_test_split(
    data_paths,
    test_size=0.25,
    stratify=data_labels
)

In [ ]:
transform = v2.Compose([
    v2.Resize((256, 256)),
    v2.CenterCrop(224),
    v2.ToTensor(),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    #v2.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [ ]:
data = datasets.ImageFolder(path, transform=transform)
data
# X = np.array([i[0] for i in data])
y = np.array([i[1] for i in data])

train_data, test_data = train_test_split(np.arange(len(data)), test_size=0.25, stratify=y, random_state=42)
train = Subset(data, train_data)
test = Subset(data, test_data)

In [ ]:
weight = compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y)
weight = torch.FloatTensor(weight).to(device)
weight

tensor([  0.5459,  11.8668,   0.8000,   4.7019,   0.3714,   5.0858,   0.4178,
          0.5055,  10.6044,   1.0627,  62.3006,   1.0906,  18.4594,  18.4594,
          4.1190,   0.2219,   1.0008,   0.4133,   1.6078, 166.1349,   0.3681,
          3.8938,   0.3861,   7.0198,   2.0260,   0.4619,  29.3179,   0.3433,
          0.3428,   1.3922,  15.5751,   6.9223,   0.4174,   7.6678,  11.0757,
          5.6001,   4.8389,   0.5683,  12.4601,   9.0619,  62.3006,   2.7536],
       device='cuda:0')

In [ ]:
trainloader = torch.utils.data.DataLoader(train, batch_size=64)
testloader = torch.utils.data.DataLoader(test, batch_size=64)

#create model

In [ ]:
model = torchvision.models.resnet34(weights='IMAGENET1K_V1')
#for param in model.parameters():
#    param.require = False
model.fc = nn.Linear(512, 42)
#model

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 95.1MB/s]


In [ ]:
#loss_fn = nn.CrossEntropyLoss(weight = weight)
#optimizer = optim.Adam(model.parameters(), lr=3e-4)
#optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.8)

In [ ]:
model = model.to(device)

In [ ]:
class fcnn(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(
          nn.Conv2d(in_channels= 3, out_channels=16, kernel_size = 4,  stride =1 , padding=1),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size= 4),
          nn.Conv2d(in_channels= 16, out_channels= 32, kernel_size= 4,  stride =1 , padding=1),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size= 4),
          nn.Conv2d(in_channels= 32, out_channels= 64, kernel_size= 4,  stride =1 , padding=1),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size= 4),
          )

        self.mlp = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=144, out_features=256, bias=True),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=256, bias=True),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=42, bias=True),
        )



    def forward(self, x):

        x = self.cnn(x)
        x= self.mlp(x)

        return x

net = fcnn().to(device)

#training model

In [ ]:
epochs = 10
size = len(trainloader.dataset)

for epoch in range(epochs):  # loop over the dataset multiple times
    print('epoch', epoch + 1, '---------------------', end='\n')

    model.train()

    for i, (X, y) in enumerate(trainloader):
        X = X.to(device)
        y = y.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(X)
        loss = loss_fn(outputs, y)
        loss.backward()
        optimizer.step()

      # print statistics
        if i % 100 == 0:
            loss, current = loss.item(), (i + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


epoch 1 ---------------------
loss: 4.120438  [   64/15699]
loss: 0.504645  [ 6464/15699]
loss: 0.435168  [12864/15699]
epoch 2 ---------------------
loss: 0.414358  [   64/15699]
loss: 0.225439  [ 6464/15699]
loss: 0.132383  [12864/15699]
epoch 3 ---------------------
loss: 0.230429  [   64/15699]
loss: 0.252279  [ 6464/15699]
loss: 0.035792  [12864/15699]
epoch 4 ---------------------
loss: 0.122867  [   64/15699]
loss: 0.115971  [ 6464/15699]
loss: 0.062463  [12864/15699]
epoch 5 ---------------------
loss: 0.157942  [   64/15699]
loss: 0.058633  [ 6464/15699]
loss: 0.063003  [12864/15699]
epoch 6 ---------------------
loss: 0.028478  [   64/15699]
loss: 0.093508  [ 6464/15699]
loss: 0.046567  [12864/15699]
epoch 7 ---------------------
loss: 0.035870  [   64/15699]
loss: 0.110619  [ 6464/15699]
loss: 0.020360  [12864/15699]
epoch 8 ---------------------
loss: 0.158103  [   64/15699]
loss: 0.015898  [ 6464/15699]
loss: 0.010398  [12864/15699]
epoch 9 ---------------------
loss: 0.08

In [ ]:
torch.save(model.state_dict(), 'model.pth')

#predict model

In [ ]:
path = 'train/simpsons_dataset/'
classes = np.array(sorted([i for i in os.listdir(path) if os.path.isdir(os.path.join(path, i))]))
classes

array(['abraham_grampa_simpson', 'agnes_skinner',
       'apu_nahasapeemapetilon', 'barney_gumble', 'bart_simpson',
       'carl_carlson', 'charles_montgomery_burns', 'chief_wiggum',
       'cletus_spuckler', 'comic_book_guy', 'disco_stu', 'edna_krabappel',
       'fat_tony', 'gil', 'groundskeeper_willie', 'homer_simpson',
       'kent_brockman', 'krusty_the_clown', 'lenny_leonard',
       'lionel_hutz', 'lisa_simpson', 'maggie_simpson', 'marge_simpson',
       'martin_prince', 'mayor_quimby', 'milhouse_van_houten',
       'miss_hoover', 'moe_szyslak', 'ned_flanders', 'nelson_muntz',
       'otto_mann', 'patty_bouvier', 'principal_skinner',
       'professor_john_frink', 'rainier_wolfcastle', 'ralph_wiggum',
       'selma_bouvier', 'sideshow_bob', 'sideshow_mel', 'snake_jailbird',
       'troy_mcclure', 'waylon_smithers'], dtype='<U24')

In [ ]:
path = 'testset/'
pred_data = torchvision.datasets.ImageFolder(path, transform=transform)
pred_dataloader = DataLoader(pred_data, batch_size=1)

In [ ]:
model.eval()
with torch.no_grad():
    y_pred = [''] * len(pred_dataloader)
    for i, (X, y) in enumerate(pred_dataloader):
        X = X.to(torch.float).to(device)
        y_pred[i] = classes[model(X)[0].argmax(0)]
y_pred

['nelson_muntz',
 'bart_simpson',
 'ned_flanders',
 'chief_wiggum',
 'apu_nahasapeemapetilon',
 'kent_brockman',
 'edna_krabappel',
 'chief_wiggum',
 'lisa_simpson',
 'kent_brockman',
 'lenny_leonard',
 'marge_simpson',
 'agnes_skinner',
 'ned_flanders',
 'moe_szyslak',
 'comic_book_guy',
 'abraham_grampa_simpson',
 'edna_krabappel',
 'agnes_skinner',
 'lisa_simpson',
 'marge_simpson',
 'nelson_muntz',
 'comic_book_guy',
 'mayor_quimby',
 'homer_simpson',
 'apu_nahasapeemapetilon',
 'krusty_the_clown',
 'krusty_the_clown',
 'ned_flanders',
 'edna_krabappel',
 'apu_nahasapeemapetilon',
 'chief_wiggum',
 'edna_krabappel',
 'ned_flanders',
 'marge_simpson',
 'abraham_grampa_simpson',
 'homer_simpson',
 'sideshow_bob',
 'lisa_simpson',
 'charles_montgomery_burns',
 'chief_wiggum',
 'agnes_skinner',
 'abraham_grampa_simpson',
 'milhouse_van_houten',
 'lenny_leonard',
 'abraham_grampa_simpson',
 'nelson_muntz',
 'comic_book_guy',
 'chief_wiggum',
 'nelson_muntz',
 'mayor_quimby',
 'charles_m

#submit

In [ ]:
sub = pd.read_csv('sample_submission.csv')
sub['Id'] = np.array(pred_data.imgs)[:, 0]
sub['Id'] = sub['Id'].apply(lambda x: x.split('/')[-1])
sub['Expected'] = y_pred
sub.to_csv('submission.csv', index=False)
sub

,Id,Expected
0,img0.jpg,nelson_muntz
1,img1.jpg,bart_simpson
2,img10.jpg,ned_flanders
3,img100.jpg,chief_wiggum
4,img101.jpg,apu_nahasapeemapetilon
...,...,...
986,img987.jpg,nelson_muntz
987,img988.jpg,charles_montgomery_burns
988,img989.jpg,charles_montgomery_burns
989,img99.jpg,chief_wiggum
